In [ ]:
import subprocess
import numpy as np
import os
import pickle as pkl
import re
import ast
import json
import math
from tqdm.notebook import tqdm

def parse_metrics(command):
    # Run the command using subprocess in Jupyter
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        # print("Output:\n", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error occurred:\n", e.stderr)

    # Split the output into lines
    output_lines = result.stdout.strip().split('\n')
    # print(output_lines)

    metrics_before, metrics_after = None, None
    # Iterate over the lines to find metrics
    for i, line in enumerate(output_lines):
        if 'Metrics before applying QB-Norm' in line:
            # The next line contains the metrics dictionary
            if i + 1 < len(output_lines):
                metrics_line = output_lines[i + 1].strip()
                try:
                    metrics_before = ast.literal_eval(metrics_line)
                except (SyntaxError, ValueError):
                    print(f"Could not parse metrics before at line {i + 1}: {metrics_line}")
        elif 'Metrics after QB-Norm' in line:
            # The next line contains the metrics dictionary
            if i + 1 < len(output_lines):
                metrics_line = output_lines[i + 1].strip()
                try:
                    metrics_after = ast.literal_eval(metrics_line)
                except (SyntaxError, ValueError):
                    print(f"Could not parse metrics after at line {i + 1}: {metrics_line}")

    # Save the metrics into a variable
    metrics = {
        'before': metrics_before,
        'after': metrics_after
    }
    return metrics

def execute_dynamic_inverted_softmax(test_file_path, adv_test_similarity_path, gt_idx_path, out_dir):
    temp_out_dir=out_dir+"/output_cache"
    # Load data from file
    data = np.load(test_file_path, allow_pickle=True)  # Load the data as a NumPy array
    # Load adversarial similarity matrices
    adv_test_similarity_matrices = np.load(adv_test_similarity_path, allow_pickle=True)
    gt_idx=np.load(gt_idx_path, allow_pickle=True)
    # Create temp directory if it doesn't exist
    os.makedirs(temp_out_dir, exist_ok=True)
    
    gt_metrics = {}
    adv_metrics = {}
    # Iterate over adversarial matrices, modify data, save with different names, and execute command
    for i, adv in enumerate(tqdm(adv_test_similarity_matrices[:100])):
        # print("Adv shape:", adv.shape)
        test_data_temp = data.copy()
        # print("Test data shape:", test_data_temp.shape)
        # print("Adv shape:", adv.shape)
        test_data_temp = np.column_stack((test_data_temp, adv.flatten()))        
        # Save the modified data to a new file with different names in temp directory
        modified_file_path = f'{temp_out_dir}/modified-test-images-texts-seed0-{i}.pkl'
        with open(modified_file_path, 'wb') as f:
            pkl.dump(test_data_temp, f)
        # print(f"Modified data saved to {modified_file_path}")


        num_queries, num_vids = test_data_temp.shape
        adv_idx = np.array([
            num_vids * (i+1) - 1 for i in range(num_queries)
        ])

        adv_idx_path = f'{temp_out_dir}/adv_idx.pkl'
        with open(adv_idx_path, 'wb') as f:
            pkl.dump(adv_idx, f)
        # print(f"adv_idx saved to {adv_idx_path}")

        modified_gt_index = gt_idx.copy()
        temp_shape = modified_gt_index.shape
        modified_gt_index = modified_gt_index.flatten() + np.arange(modified_gt_index.size)
        modified_gt_index = modified_gt_index.reshape(temp_shape)

        modified_gt_index_path=f'{temp_out_dir}/modified_gt_idx-{i}.pkl'
        with open(modified_gt_index_path, 'wb') as f:
            pkl.dump(modified_gt_index, f)
        # print(f"modified_gt_idx saved to {modified_gt_index_path}")

        # Define the command and arguments for each modified data file
        gt_command = [
            'python', 'dynamic_inverted_softmax.py',
            '--sims_test_path', modified_file_path,
            '--gt_idx_path', modified_gt_index_path
        ]

        adv_command = [
            'python', 'dynamic_inverted_softmax.py',
            '--sims_test_path', modified_file_path,
            '--gt_idx_path', adv_idx_path
        ]
        
        # Run the command
        gt_metrics[i]=parse_metrics(gt_command)
        print(gt_metrics[i])

        adv_metrics[i]=parse_metrics(adv_command)
        print(adv_metrics[i])
        
        # save the metrics to a file
        with open(f'{out_dir}/transfer_openclip_300_gt_metrics.pkl', 'wb') as f:
            pkl.dump(gt_metrics, f)
        with open(f'{out_dir}/transfer_openclip_300_adv_metrics.pkl', 'wb') as f:
            pkl.dump(adv_metrics, f)

        # Remove the modified data files
        # os.remove(modified_file_path)
        
    return gt_metrics, adv_metrics


In [12]:
out_dir = '../outputs/mscoco/imagebind/naive'
test_file_path = f'{out_dir}/test_similarity_matrix.pkl'
adv_test_similarity_path = f'{out_dir}/transfer_openclip_300_adv_test_similarity_matrix.pkl'
gt_idx_path='../outputs/mscoco/gt_idx.pkl'
os.makedirs(out_dir, exist_ok=True)

# print the shape of the similarity matrices
test_data = np.load(test_file_path, allow_pickle=True)
adv_test_similarity_matrices = np.load(adv_test_similarity_path, allow_pickle=True)
gt_idx=np.load(gt_idx_path, allow_pickle=True)
print("Test data shape:", test_data.shape)
print("Adv test similarity matrices shape:", adv_test_similarity_matrices.shape)
print("gt_idx shape:", gt_idx.shape)

# Define the command and arguments
command = [
    'python', 'dynamic_inverted_softmax.py',
    '--sims_test_path', test_file_path,
    '--gt_idx_path', gt_idx_path
]

# Parse the metrics
metrics = parse_metrics(command)
print("Mertrics for original data:")
print(metrics)

gt_metrics, adv_metrics = execute_dynamic_inverted_softmax(test_file_path, adv_test_similarity_path, gt_idx_path, out_dir)

Test data shape: (25000, 5000)
Adv test similarity matrices shape: (100, 25000)
gt_idx shape: (5000, 5)
Mertrics for original data:
{'before': {'R1': 48.4, 'R3': 65.6, 'R5': 72.7, 'R10': 81.0, 'R50': 95.3, 'MedR': 2.0, 'MeanR': 13.8, 'geometric_mean_R1-R5-R10': 65.8, 'MeanA': 0.311}, 'after': None}


  0%|          | 0/100 [00:00<?, ?it/s]

{'before': {'R1': 13.3, 'R3': 59.8, 'R5': 69.7, 'R10': 79.8, 'R50': 95.1, 'MedR': 3.0, 'MeanR': 14.7, 'geometric_mean_R1-R5-R10': 41.9, 'MeanA': 0.311}, 'after': None}
{'before': {'R1': 81.9, 'R3': 96.3, 'R5': 97.7, 'R10': 98.8, 'R50': 99.8, 'MedR': 1.0, 'MeanR': 1.7, 'geometric_mean_R1-R5-R10': 92.5, 'MeanA': 0.38}, 'after': None}
{'before': {'R1': 19.9, 'R3': 60.0, 'R5': 69.7, 'R10': 79.8, 'R50': 95.1, 'MedR': 3.0, 'MeanR': 14.6, 'geometric_mean_R1-R5-R10': 48.0, 'MeanA': 0.311}, 'after': None}
{'before': {'R1': 71.3, 'R3': 94.1, 'R5': 96.7, 'R10': 98.5, 'R50': 99.9, 'MedR': 1.0, 'MeanR': 1.8, 'geometric_mean_R1-R5-R10': 87.9, 'MeanA': 0.358}, 'after': None}
{'before': {'R1': 18.3, 'R3': 60.1, 'R5': 69.7, 'R10': 79.8, 'R50': 95.1, 'MedR': 3.0, 'MeanR': 14.6, 'geometric_mean_R1-R5-R10': 46.7, 'MeanA': 0.311}, 'after': None}
{'before': {'R1': 73.6, 'R3': 94.0, 'R5': 96.3, 'R10': 98.1, 'R50': 99.8, 'MedR': 1.0, 'MeanR': 2.0, 'geometric_mean_R1-R5-R10': 88.6, 'MeanA': 0.362}, 'after': No

In [ ]:

# out_dir='../outputs/mscoco/audioclip_partial'
# # load the metrics from the file
# with open(f'{out_dir}/transfer_openclip_rn50_300_gt_metrics.pkl', 'rb') as f:
#     temp_gt_metrics = pkl.load(f)
# with open(f'{out_dir}/transfer_openclip_rn50_300_adv_metrics.pkl', 'rb') as f:
#     temp_adv_metrics = pkl.load(f)

def post_analysis(data):
    # Initialize dictionaries to hold lists of metric values across entries
    metrics_before = {key: [] for key in data[0]['before']}

    # Collect all metric values for each metric type across all entries
    for entry in data.values():
        for metric, value in entry['before'].items():
            metrics_before[metric].append(value)

    # Function to calculate mean and standard deviation
    def calculate_stats(metrics_dict):
        stats = {}
        for metric, values in metrics_dict.items():
            avg = round(np.mean(values), 1)
            std_dev = round(np.std(values), 1)
            stats[metric] = {'average': avg, 'std_dev': std_dev}
        return stats

    # Calculate stats for 'before' and 'after'
    before_stats = calculate_stats(metrics_before)

    return before_stats

print("gt stats:", post_analysis(gt_metrics))
print("adv stats:", post_analysis(adv_metrics))


gt stats: {'R1': {'average': 17.1, 'std_dev': 3.7}, 'R3': {'average': 60.0, 'std_dev': 0.2}, 'R5': {'average': 69.7, 'std_dev': 0.1}, 'R10': {'average': 79.8, 'std_dev': 0.0}, 'R50': {'average': 95.1, 'std_dev': 0.0}, 'MedR': {'average': 2.7, 'std_dev': 0.4}, 'MeanR': {'average': 14.6, 'std_dev': 0.1}, 'geometric_mean_R1-R5-R10': {'average': 45.4, 'std_dev': 3.2}, 'MeanA': {'average': 0.3, 'std_dev': 0.0}}
adv stats: {'R1': {'average': 75.4, 'std_dev': 6.2}, 'R3': {'average': 94.5, 'std_dev': 2.2}, 'R5': {'average': 96.7, 'std_dev': 1.3}, 'R10': {'average': 98.3, 'std_dev': 0.7}, 'R50': {'average': 99.8, 'std_dev': 0.1}, 'MedR': {'average': 1.0, 'std_dev': 0.0}, 'MeanR': {'average': 2.0, 'std_dev': 0.4}, 'geometric_mean_R1-R5-R10': {'average': 89.4, 'std_dev': 3.1}, 'MeanA': {'average': 0.4, 'std_dev': 0.0}}


In [17]:
out_dir='../outputs/mscoco/audioclip_partial'
# load the metrics from the file
with open(f'{out_dir}/transfer_openclip_rn50_300_gt_metrics.pkl', 'rb') as f:
    temp_gt_metrics = pkl.load(f)
with open(f'{out_dir}/transfer_openclip_rn50_300_adv_metrics.pkl', 'rb') as f:
    temp_adv_metrics = pkl.load(f)
print("gt stats:", post_analysis(temp_gt_metrics))
print("adv stats:", post_analysis(temp_adv_metrics))

gt stats: {'R1': {'average': 18.8, 'std_dev': 0.0}, 'R3': {'average': 32.3, 'std_dev': 0.0}, 'R5': {'average': 39.5, 'std_dev': 0.1}, 'R10': {'average': 50.7, 'std_dev': 0.0}, 'R50': {'average': 80.1, 'std_dev': 0.0}, 'MedR': {'average': 10.0, 'std_dev': 0.0}, 'MeanR': {'average': 47.9, 'std_dev': 0.1}, 'geometric_mean_R1-R5-R10': {'average': 33.5, 'std_dev': 0.0}, 'MeanA': {'average': 0.2, 'std_dev': 0.0}}
adv stats: {'R1': {'average': 0.2, 'std_dev': 0.3}, 'R3': {'average': 0.5, 'std_dev': 0.7}, 'R5': {'average': 0.8, 'std_dev': 1.1}, 'R10': {'average': 1.5, 'std_dev': 1.9}, 'R50': {'average': 7.1, 'std_dev': 6.6}, 'MedR': {'average': 520.3, 'std_dev': 343.8}, 'MeanR': {'average': 690.2, 'std_dev': 359.1}, 'geometric_mean_R1-R5-R10': {'average': 0.6, 'std_dev': 0.8}, 'MeanA': {'average': 0.1, 'std_dev': 0.0}}


In [8]:
import pickle
import numpy as np

# Load the matrix
with open(test_file_path, 'rb') as f:
    matrix = pickle.load(f)

# Transpose the matrix
transposed_matrix = np.transpose(matrix)

# Save the transposed matrix back to the original path
with open(test_file_path, 'wb') as f:
    pickle.dump(transposed_matrix, f)

In [ ]:
import pickle
import numpy as np

# Load the matrix
with open(test_file_path, 'rb') as f:
    matrix = pickle.load(f)

# Transpose the matrix
transposed_matrix = np.transpose(matrix)

# Save the transposed matrix back to the original path
with open(test_file_path, 'wb') as f:
    pickle.dump(transposed_matrix, f)

In [7]:
out_dir='../outputs/mscoco/imagebind/naive'
# load the metrics from the file
with open(f'{out_dir}/gt_metrics.pkl', 'rb') as f:
    temp_gt_metrics = pkl.load(f)
with open(f'{out_dir}/adv_metrics.pkl', 'rb') as f:
    temp_adv_metrics = pkl.load(f)

def post_analysis(data):
    # Initialize dictionaries to hold lists of metric values across entries
    metrics_before = {key: [] for key in data[0]['before']}
    metrics_after = {key: [] for key in data[0]['after']}

    # Collect all metric values for each metric type across all entries
    for entry in data.values():
        for metric, value in entry['before'].items():
            metrics_before[metric].append(value)
        for metric, value in entry['after'].items():
            metrics_after[metric].append(value)

    # Function to calculate mean and standard deviation
    def calculate_stats(metrics_dict):
        stats = {}
        for metric, values in metrics_dict.items():
            avg = round(np.mean(values), 1)
            std_dev = round(np.std(values), 1)
            stats[metric] = {'average': avg, 'std_dev': std_dev}
        return stats

    # Calculate stats for 'before' and 'after'
    before_stats = calculate_stats(metrics_before)
    after_stats = calculate_stats(metrics_after)

    return before_stats, after_stats

print("gt stats:", post_analysis(temp_gt_metrics))
print("adv stats:", post_analysis(temp_adv_metrics))



TypeError: 'NoneType' object is not iterable

In [8]:
temp_gt_metrics

{0: {'before': {'R1': 9.9,
   'R5': 69.6,
   'R10': 79.8,
   'R50': 95.1,
   'MedR': 3.0,
   'MeanR': 14.7,
   'geometric_mean_R1-R5-R10': 38.1,
   'MeanA': 0.311},
  'after': None},
 1: {'before': {'R1': 8.8,
   'R5': 69.6,
   'R10': 79.8,
   'R50': 95.1,
   'MedR': 3.0,
   'MeanR': 14.7,
   'geometric_mean_R1-R5-R10': 36.6,
   'MeanA': 0.311},
  'after': None}}